In [1]:
# get the environment
import sys
print(sys.version)
print(sys.executable)
import os
os.getcwd()

3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
C:\Users\bbutler\Documents\Anaconda3\envs\timeseries\python.exe


'C:\\Users\\bbutler\\Documents\\DigitalAnalytics'

In [2]:
# import base libraries
from IPython.core.display import display, HTML
from dateutil import relativedelta
import pandas as pd
import numpy as np
import pyodbc
import re
import statistics as stat
import datetime as dt
from datetime import timedelta

# set options
display(HTML("<style>.container {width:90% !important;} </style>"))
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.2f}'.format

In [4]:
# change the dir
os.chdir('C:\\Users\\bbutler\\Documents\\DigitalAnalytics\\SocialDataDownloads')

In [23]:
social = pd.read_csv('PLoanSocialData.csv',low_memory=False)
social.head()

,Reporting Starts,Reporting Ends,Campaign Name,Ad Set Name,Ad Name,Platform,Placement,Device Platform,Impression Device,Amount Spent (USD),Impressions,Link Clicks,Landing Page Views,App Starts,App Completes
0,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPhone,$7.31,524,2.00,2.00,nan,nan
1,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPad,$0.04,1,nan,nan,nan,nan
2,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,Android Smartphone,$5.09,123,1.00,1.00,nan,1.00
3,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPhone,$8.33,244,nan,nan,nan,nan
4,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPad,$0.02,1,nan,nan,nan,nan


In [24]:
social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Reporting Starts      415 non-null    object 
 1   Reporting Ends        415 non-null    object 
 2   Campaign Name         415 non-null    object 
 3   Ad Set Name           415 non-null    object 
 4   Ad Name               415 non-null    object 
 5   Platform              415 non-null    object 
 6   Placement             415 non-null    object 
 7   Device Platform       415 non-null    object 
 8   Impression Device     415 non-null    object 
 9    Amount Spent (USD)   415 non-null    object 
 10  Impressions           415 non-null    int64  
 11  Link Clicks           76 non-null     float64
 12  Landing Page Views    71 non-null     float64
 13  App Starts            16 non-null     float64
 14  App Completes         22 non-null     float64
dtypes: float64(4), int64(1)

In [25]:
# create date
# pd.to_datetime, format = '%Y-%m-%d'
social['Date'] = pd.to_datetime(social['Reporting Starts'], format = '%m/%d/%Y')
social.head()

,Reporting Starts,Reporting Ends,Campaign Name,Ad Set Name,Ad Name,Platform,Placement,Device Platform,Impression Device,Amount Spent (USD),Impressions,Link Clicks,Landing Page Views,App Starts,App Completes,Date
0,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPhone,$7.31,524,2.00,2.00,nan,nan,2021-01-10
1,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPad,$0.04,1,nan,nan,nan,nan,2021-01-10
2,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,Android Smartphone,$5.09,123,1.00,1.00,nan,1.00,2021-01-10
3,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPhone,$8.33,244,nan,nan,nan,nan,2021-01-10
4,1/10/2021,1/10/2021,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPad,$0.02,1,nan,nan,nan,nan,2021-01-10


In [26]:
# drop the reporting starts fields
social = social.drop('Reporting Starts', axis = 1)
social = social.drop('Reporting Ends', axis = 1)
social.head()

,Campaign Name,Ad Set Name,Ad Name,Platform,Placement,Device Platform,Impression Device,Amount Spent (USD),Impressions,Link Clicks,Landing Page Views,App Starts,App Completes,Date
0,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPhone,$7.31,524,2.00,2.00,nan,nan,2021-01-10
1,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPad,$0.04,1,nan,nan,nan,nan,2021-01-10
2,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,Android Smartphone,$5.09,123,1.00,1.00,nan,1.00,2021-01-10
3,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPhone,$8.33,244,nan,nan,nan,nan,2021-01-10
4,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPad,$0.02,1,nan,nan,nan,nan,2021-01-10


In [27]:
social.columns

Index(['Campaign Name', 'Ad Set Name', 'Ad Name', 'Platform', 'Placement',
       'Device Platform', 'Impression Device', ' Amount Spent (USD) ',
       'Impressions', 'Link Clicks', 'Landing Page Views', 'App Starts',
       'App Completes', 'Date'],
      dtype='object')

In [28]:
# clean up the amount spent 
social[' Amount Spent (USD) '] = social[' Amount Spent (USD) '].str.replace(',','')

In [31]:
# clean up the amount spent 
social[' Amount Spent (USD) '] = social[' Amount Spent (USD) '].str.replace('-','0')
social[' Amount Spent (USD) '] = social[' Amount Spent (USD) '].str.replace(',','')
social[' Amount Spent (USD) '] = social[' Amount Spent (USD) '].str.replace('$','')
social[' Amount Spent (USD) '] = social[' Amount Spent (USD) '].astype(float)
# rename it in place
social.rename(columns={' Amount Spent (USD) ':'Spend'}, inplace=True)
social.head()


,Campaign Name,Ad Set Name,Ad Name,Platform,Placement,Device Platform,Impression Device,Spend,Impressions,Link Clicks,Landing Page Views,App Starts,App Completes,Date
0,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPhone,7.31,524,2.00,2.00,nan,nan,2021-01-10
1,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,iPad,0.04,1,nan,nan,nan,nan,2021-01-10
2,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Instagram Stories,Mobile App,Android Smartphone,5.09,123,1.00,1.00,nan,1.00,2021-01-10
3,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPhone,8.33,244,nan,nan,nan,nan,2021-01-10
4,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Eastern Bank_DR_Facebook_Personal Loans_Jan Ca...,Instagram,Feed: News Feed,Mobile App,iPad,0.02,1,nan,nan,nan,nan,2021-01-10


In [32]:
social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Campaign Name       415 non-null    object        
 1   Ad Set Name         415 non-null    object        
 2   Ad Name             415 non-null    object        
 3   Platform            415 non-null    object        
 4   Placement           415 non-null    object        
 5   Device Platform     415 non-null    object        
 6   Impression Device   415 non-null    object        
 7   Spend               415 non-null    float64       
 8   Impressions         415 non-null    int64         
 9   Link Clicks         76 non-null     float64       
 10  Landing Page Views  71 non-null     float64       
 11  App Starts          16 non-null     float64       
 12  App Completes       22 non-null     float64       
 13  Date                415 non-null    datetime64[ns]

In [35]:
# convert link clicks, landing page views, app starts, app completes
social.iloc[:,[9,10,11,12]] = social.iloc[:,[9,10,11,12]].astype('int')
social.info()

ValueError: Cannot convert non-finite values (NA or inf) to integer